In [10]:
import pandas as pd

# Ruta al archivo 
archivo = "/Users/severinodonate/Downloads/planet_osm_polygon_202505060635.csv"


# Leer el CSV con el separador correcto
df = pd.read_csv(archivo, sep=";", quotechar='"')

# Mostrar las primeras filas
df.head()


/var/folders/59/2dj5q51d63d6npy9bjv7ms0w0000gn/T/ipykernel_4639/1119506526.py:8: DtypeWarning: Columns (2,3,4,7,11,13,14,16,17,19,20,21,22,23,26,28,29,33,36,41,43,48,49,50,51,54,56,58,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, sep=";", quotechar='"')


,osm_id,access,addr:housename,addr:housenumber,addr:interpolation,admin_level,aerialway,aeroway,amenity,area,...,tunnel,water,waterway,wetland,width,wood,z_order,way_area,tags,way
0,687265667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,61.8973,NaN,POLYGON ((-19486269.20798441 -2408171.71697459...
1,779567922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,4648.7000,"""name:en""=>""Tia Seu ancient mound"", ""wikidata""...",POLYGON ((-19183097.98868702 -1543861.79670201...
2,346277921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,17846.4000,"""alt_name""=>""Kolo Nui"", ""wikidata""=>""Q7679508""...",POLYGON ((-19615560.01718381 -1498294.24405601...
3,1177174544,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,254.4990,"""archaeological_site""=>""settlement""",POLYGON ((-16807136.510872487 -1899663.6392141...
4,216130025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,marae,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,483.3440,NaN,POLYGON ((-16849369.455887094 -1901704.8375546...


In [11]:
import re

def parse_tags(tag_str):
    """
    Convierte un string de tags tipo 'key'=>"value" en un diccionario Python
    """
    if not isinstance(tag_str, str) or tag_str.strip() == "":
        return {}

    # Elimina comillas triples y extrae pares key=>value
    tag_pairs = re.findall(r'"([^"]+)"\s*=>\s*"([^"]*)"', tag_str)
    return {key: value for key, value in tag_pairs}


In [12]:
ejemplo = df['tags'].dropna().iloc[0]
print("Original:", ejemplo)
print("Parseado:", parse_tags(ejemplo))


Original: "name:en"=>"Tia Seu ancient mound", "wikidata"=>"Q7259289"
Parseado: {'name:en': 'Tia Seu ancient mound', 'wikidata': 'Q7259289'}


In [13]:
df['tags_dict'] = df['tags'].apply(parse_tags)

In [14]:
# Aplicar la función a toda la columna
df['tags_dict'] = df['tags'].apply(parse_tags)

# Mostrar solo las columnas 'tags' y 'tags_dict' para comparar
df[['tags', 'tags_dict']].head()

,tags,tags_dict
0,NaN,{}
1,"""name:en""=>""Tia Seu ancient mound"", ""wikidata""...","{'name:en': 'Tia Seu ancient mound', 'wikidata..."
2,"""alt_name""=>""Kolo Nui"", ""wikidata""=>""Q7679508""...","{'alt_name': 'Kolo Nui', 'wikidata': 'Q7679508..."
3,"""archaeological_site""=>""settlement""",{'archaeological_site': 'settlement'}
4,NaN,{}


In [15]:
# Expandir los diccionarios en columnas
tags_expandidos = pd.json_normalize(df['tags_dict'])

# Unir las nuevas columnas al dataframe original
df_expandido = pd.concat([df.drop(columns=['tags_dict']), tags_expandidos], axis=1)

# Mostrar las primeras filas del nuevo dataframe
df_expandido.head()


,osm_id,access,addr:housename,addr:housenumber,addr:interpolation,admin_level,aerialway,aeroway,amenity,area,...,protect_id,name:Nawat,name:source,operator:wikipedia,lacounty:ain,lacounty:bld_id,historic_2,ref:borden,ref:nhl,name:haw
0,687265667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,779567922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,346277921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1177174544,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,216130025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,marae,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
print("Número de columnas:", df_expandido.shape[1])

Número de columnas: 1244


In [17]:
df_expandido.columns.tolist()

['osm_id',
 'access',
 'addr:housename',
 'addr:housenumber',
 'addr:interpolation',
 'admin_level',
 'aerialway',
 'aeroway',
 'amenity',
 'area',
 'barrier',
 'bicycle',
 'brand',
 'bridge',
 'boundary',
 'building',
 'construction',
 'covered',
 'culvert',
 'cutting',
 'denomination',
 'disused',
 'embankment',
 'foot',
 'generator:source',
 'harbour',
 'highway',
 'historic',
 'horse',
 'intermittent',
 'junction',
 'landuse',
 'layer',
 'leisure',
 'lock',
 'man_made',
 'military',
 'motorcar',
 'name',
 'natural',
 'office',
 'oneway',
 'operator',
 'place',
 'population',
 'power',
 'power_source',
 'public_transport',
 'railway',
 'ref',
 'religion',
 'route',
 'service',
 'shop',
 'sport',
 'surface',
 'toll',
 'tourism',
 'tower:type',
 'tracktype',
 'tunnel',
 'water',
 'waterway',
 'wetland',
 'width',
 'wood',
 'z_order',
 'way_area',
 'tags',
 'way',
 'name:en',
 'wikidata',
 'alt_name',
 'wikipedia',
 'archaeological_site',
 'old_name',
 'alt_name:ty',
 'wheelchair',
 'f

In [18]:
# Guardar el dataframe en un nuevo archivo CSV
df_expandido.to_csv("/Users/severinodonate/Downloads/planet_osm_polygons_expandido.csv", index=False, sep=';', quoting=1)

In [19]:
# Número total de filas
total_filas = len(df_expandido)

# Crear resumen de cobertura por variable
resumen = (
    df_expandido.notna()
    .sum()
    .reset_index()
    .rename(columns={"index": "variable", 0: "valores"})
)

# Calcular porcentaje de cobertura
resumen["porcentaje"] = (resumen["valores"] / total_filas * 100).round(2)

# Ordenar por porcentaje descendente
resumen = resumen.sort_values(by="porcentaje", ascending=False).reset_index(drop=True)

resumen


,variable,valores,porcentaje
0,osm_id,57777,100.0
1,z_order,57777,100.0
2,way,57777,100.0
3,way_area,57777,100.0
4,historic,56392,97.6
...,...,...,...
1239,refitted,1,0.0
1240,sinkhole,1,0.0
1241,dataset,1,0.0
1242,addr:unit,1,0.0


In [20]:
resumen.to_csv("/Users/severinodonate/Downloads/resumen_variables_poligonos.csv", index=False)

In [21]:
import pandas as pd

# Cargar el archivo original
archivo = "/Users/severinodonate/Downloads/planet_osm_polygon_202505060635.csv"
df = pd.read_csv(archivo, sep=";", quotechar='"')

# Asegurarse de que la columna se llama 'tags'
columna_tags = [col for col in df.columns if 'tags' in col.lower()][0]

# Contar cuántas veces aparece 'wikidata' en esa columna
conteo = df[columna_tags].fillna('').str.contains('wikidata').sum()

print(f"La palabra 'wikidata' aparece en {conteo} filas de la columna '{columna_tags}'.")



La palabra 'wikidata' aparece en 8125 filas de la columna 'tags'.


/var/folders/59/2dj5q51d63d6npy9bjv7ms0w0000gn/T/ipykernel_4639/2417730505.py:5: DtypeWarning: Columns (2,3,4,7,11,13,14,16,17,19,20,21,22,23,26,28,29,33,36,41,43,48,49,50,51,54,56,58,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, sep=";", quotechar='"')
